In [2]:
from bs4 import BeautifulSoup
from requests import request
from datetime import datetime
from requests import codes
from html2text import html2text
from json import dumps, dump
import sqlalchemy as sa
import yaml
import logging
import pandas as pd
import re

In [3]:
class boardgamescraper(object): 

    def __init__(self, configLocation:str, setupDbConnection:bool):

        with open(configLocation, "r") as inFile:
            try:
                self.config = yaml.safe_load(inFile)
                logging.info('Loaded Config')
            except yaml.YAMLError as exc:
                logging.error(exc)

        if setupDbConnection:
            self.setupSQLServer(self.config['DB_driver'],self.config['DB_host'],self.config['DB_database'])
        else:
            self.SQLConnectionReady = False
    
    """ Following Function are for Webscraping boardgame website"""

    def getBoardgameIds(self, page=1):
        url = f'https://boardgamegeek.com/browse/boardgame/page/{page}'

        response = request('GET',url)
        if response.status_code == codes.ok:
            parsed_html = BeautifulSoup(response.text, 'lxml')
            data = []
            for row in parsed_html.find_all( 'a',{"class": "primary"}):
                dict = {}
                dict['name']=row.text
                id = re.split('/',row['href'])[2]
                dict['objectid'] = id
                dict['link']=row['href']
                data.append(dict)
            return data
        else:
            return None

    def getBoardgameDetails(self, objectid):
        params = {'nosession': 1, 
            'objectid': objectid,
            'objecttype':'thing',
            'subtype':'boardgame',
            'type':'things'}

        url = f'https://api.geekdo.com/api/geekitems'


        response = request("GET", url,params=params)
        if response.status_code == codes.ok:
            data = response.json()['item']
            self.outputRawLocal(data,'dataDumps\Details',objectid)
            return data
        else:
            return None

    def getBoardgameStats(self, objectid):
        params = {'nosession': 1, 
            'objectid': objectid,
            'objecttype':'thing',
            'subtype':'boardgame',
            'type':'things'}

        url = f'https://api.geekdo.com/api/dynamicinfo'


        response = request("GET", url,params=params)
        if response.status_code == codes.ok:
            data = response.json()['item']
            data['objectid'] = objectid
            self.outputRawLocal(data,'dataDumps\Stats',objectid)
            return data
        else:
            return None

    """ Following Function are for Outputing data to different sources"""

    def setupSQLServer(self, driver, server,database):
        connection_string = (
            f"DRIVER={driver};"
            f"SERVER={server};"
            f"DATABASE={database};"
            r"Trusted_Connection=Yes;"
        )
        sqlalchemy_url = "mssql+pyodbc:///?odbc_connect=" + connection_string
        self.engine = sa.create_engine(sqlalchemy_url, fast_executemany=True)
        self.SQLConnectionReady = True
    
    def SQLOutputUpsert(self, df_update, targetTable, idColumns):
        logging.info(f'Writing to {targetTable} table')
        if not self.SQLConnectionReady:
            logging.error('SQL Server connection not setup')
            return False

        sourceColumnNames = df_update.columns
        with self.engine.begin() as conn:
            #Get columns in table trying to write to
            SQLREQUEST = conn.execute(sa.text("SELECT TOP 0 * from {}".format(targetTable)))
            targetColumnNames = [col[0] for col in SQLREQUEST.cursor.description]
            
            #find common columns between target table and source data and check id fields in 
            columns = list(set(targetColumnNames).intersection(sourceColumnNames))
            if not columns: 
                logging.error('No common columns')
                return False
            
            #check if id columns exist
            idCheck = [x in columns for x in idColumns]
            if False in idCheck:
                logging.error('ID column not in Target/Source')
                return False
            #probably should put in error check to notify which columns mismatch
            
            try:
                #Load Data into temporary table
                df_update.to_sql("DataLoad", conn, index=False, if_exists='replace')


                #Create Update query based off columns
                updateQuery = 'UPDATE t \nSET'
                # for each column ad a line to query to set it
                for col in columns:
                    updateQuery = updateQuery + '\n\tt.{0} = s.{0},'.format(col)
                #Get rid of last comma and add on tabe/join section
                updateQuery = updateQuery[:-1] + '\nFROM {} t \nJOIN DataLoad s ON '.format(targetTable)
                #create join conditions for each of the ID fields
                for col in idColumns:
                    updateQuery = updateQuery + '\n\tt.{0} = s.{0} and'.format(col)
                updateQuery = updateQuery[:-3] #Get rid of AND for last one


                #create Insert query based off columns
                #Get Columns to be updated
                insertColumns = ''
                for col in columns:
                    insertColumns = insertColumns + '{},'.format(col)
                insertColumns = insertColumns[:-1]
                #Create Query
                insertQuery = 'INSERT INTO {0} ({1}) \n'\
                    'SELECT '\
                    '\n\t{1} '\
                    '\nFROM DataLoad s '\
                    '\nWHERE NOT EXISTS '\
                    '\n\t(SELECT NULL'\
                    '\n\t FROM {0} t'\
                    '\n\tWhere '.format(targetTable,insertColumns)
                #add id conditions
                for col in idColumns:
                    insertQuery = insertQuery + '\n\t\tt.{0} = s.{0} and'.format(col)
                insertQuery = insertQuery[:-3] +')' #Get rid of AND for last one

                mainQuery = """
                SET NOCOUNT ON;
                DECLARE @rows_updated INT = 0;
                DECLARE @rows_inserted INT = 0;
                
                {};
                SELECT @rows_updated = @@ROWCOUNT;
                
                {};
                SELECT @rows_inserted = @@ROWCOUNT;
                
                SELECT @rows_updated AS rows_updated, @rows_inserted AS rows_inserted;
                """.format(updateQuery,insertQuery)
                
                #Run Query
                result = conn.execute(sa.text(mainQuery)).fetchone()
                logging.info(f"{result[0]} row(s) updated, {result[1]} row(s) inserted")
            finally:
                #drop the temporary table
                conn.execute(sa.text("DROP TABLE IF EXISTS DataLoad"))
            
            return True

    def SQLOutput(self, df_update, targetTable, if_exists ='fail'):
        logging.info(f'Writing to {targetTable} table')
        if not self.SQLConnectionReady:
            logging.error('SQL Server connection not setup')
            return False

        with self.engine.begin() as conn:
            df_update.to_sql(targetTable, conn, index=False, if_exists=if_exists)

    def outputRawLocal(self,data,location, name):
        with open(f"{location}\{name}.json", "w") as outfile:
            dump(data, outfile)
    
    """ Following Function are for Parse response data"""

    def ParseData(self, details, stats):
        links = []
        today = datetime.today()
        for detail in details:
            if 'links' in detail.keys():
                link = detail['links']
                link['gameid'] = detail['objectid']
                links.append(link)

        #parse the all the links for each board game 
        #this is tall data that has many descriptive things like developer/artist
        boardgame_links =  pd.json_normalize(links, sep='_')
        boardgame_links = boardgame_links.melt(var_name='link_type',id_vars='gameid')
        boardgame_links = boardgame_links.explode('value',ignore_index=True)
        boardgame_links.dropna(axis=0, subset =['value'], inplace=True)
        boardgame_links.reset_index(drop=True,inplace=True)
        links_expanded =  pd.DataFrame(boardgame_links['value'].values.tolist())
        boardgame_links.drop(labels='value',axis=1,inplace=True)
        boardgame_links = pd.concat([boardgame_links, links_expanded], axis=1)
        boardgame_links = boardgame_links.convert_dtypes()

        #Parse the response for the board game details
        boardgame_details =  pd.json_normalize(details, sep='_',max_level=1)
        #dict of columns to keep and the data types they should be
        columns = {'objectid':'Int64', 'label':'string', 'href':'string',
        'name':'string', 'yearpublished':'Int32', 'minplayers':'Int32', 
        'maxplayers':'Int32', 'minplaytime':'Int32', 'maxplaytime':'Int32',
        'minage':'Int32', 'short_description':'string', 'description':'string',
        'wiki':'string', 'imageurl':'string', 'topimageurl':'string',
        'website_url':'string', 'website_title':'string', 'images_thumb':'string',
        'images_square200':'string'}
        columns = {k:v for k,v in columns.items() if k in boardgame_details.columns}
        boardgame_details = boardgame_details[columns]
        boardgame_details = boardgame_details.convert_dtypes()
        boardgame_details = boardgame_details.astype(columns, errors='ignore')

        #Clean up html tags in the some text fields
        boardgame_details['wiki'] = boardgame_details.apply(lambda x: html2text(x['wiki']),axis=1)
        boardgame_details['description'] = boardgame_details.apply(lambda x: html2text(x['description']),axis=1)


        #Parse out the rank information from the stats data stream
        columns_stats_rank = {'objectid':'Int64', 'rank':'Int32', 'baverage':'Float32'}
        stats_rank = [x for x in stats if 'rankinfo' in x.keys()]
        boardgame_stats_rank =  pd.json_normalize(stats_rank, sep='_', record_path='rankinfo', meta='objectid')
        boardgame_stats_rank = boardgame_stats_rank.convert_dtypes()
        boardgame_stats_rank = boardgame_stats_rank.astype(columns_stats_rank, errors='ignore')
        

        #parse out the user poll best player count from stats
        stats_polls_best = [x for x in stats if 'best' in x['polls']['userplayers'].keys()]
        boardgame_stats_polls_best =  pd.json_normalize(stats_polls_best, sep='_', record_path=['polls','userplayers','best'], meta='objectid')
        columns_polls_best = {'min': 'players_best_min','max': 'players_best_max'}
        boardgame_stats_polls_best.rename(columns=columns_polls_best, inplace=True)
        boardgame_stats_polls_best.drop_duplicates(subset='objectid', inplace=True)
        boardgame_stats_polls_best = boardgame_stats_polls_best.convert_dtypes()
        boardgame_stats_polls_best = boardgame_stats_polls_best.astype({'objectid': 'Int64'}, errors='ignore')

        #parse out the user poll recommended player count from stats
        stats_polls_recommended = [x for x in stats if 'recommended' in x['polls']['userplayers'].keys()]
        boardgame_stats_polls_recommended =  pd.json_normalize(stats_polls_recommended, sep='_', record_path=['polls','userplayers','recommended'], meta='objectid')
        columns_polls_recommended = {'min': 'players_recommended_min','max': 'players_recommended_max'}
        boardgame_stats_polls_recommended.rename(columns=columns_polls_recommended, inplace=True)
        boardgame_stats_polls_recommended.drop_duplicates(subset='objectid', inplace=True)
        boardgame_stats_polls_recommended = boardgame_stats_polls_recommended.convert_dtypes()
        boardgame_stats_polls_recommended = boardgame_stats_polls_recommended.astype({'objectid': 'Int64'}, errors='ignore')

        #parse the remaining stats data removing subobjects previously parsed
        boardgame_stats =  pd.json_normalize(stats, sep='_')
        drop_columns = ['rankinfo','commercelinks','polls_userplayers_best','polls_userplayers_recommended','has_ggs_link','polls_subdomain']
        drop_columns = [x for x in drop_columns if x in boardgame_stats.columns]
        boardgame_stats.drop(drop_columns,axis=1,inplace=True)
        poll_columns = {x: re.sub('polls_','',x) for x in boardgame_stats.columns if 'polls_' in x}
        stat_columns = {x: re.sub('stats_','',x) for x in boardgame_stats.columns if 'stats_' in x}
        relatedcount_columnns = {x: re.sub('relatedcounts_','',x)+'_count' for x in boardgame_stats.columns if 'relatedcounts_' in x}
        stats_column_renames = {**poll_columns, **stat_columns, **relatedcount_columnns}
        boardgame_stats.rename(columns=stats_column_renames, inplace=True)

        boardgame_stats = boardgame_stats.convert_dtypes()
        boardgame_stats_dtypes = {'objectid': 'Int64', 'userplayers_totalvotes': 'Int32', 
        'boardgameweight_averageweight' : 'Float32', 'boardgameweight_votes': 'Int32',
            'usersrated': 'Int32', 'average' : 'Float32', 'baverage' : 'Float32', 'stddev' : 'Float32', 'avgweight' : 'Float32',
            'numweights': 'Int32', 'numgeeklists': 'Int32', 'numtrading': 'Int32', 'numwanting': 'Int32', 'numwish': 'Int32',
            'numowned': 'Int32', 'numprevowned': 'Int32', 'numcomments': 'Int32', 'numwishlistcomments': 'Int32',
            'numhasparts': 'Int32', 'numwantparts': 'Int32', 'views': 'Int32',  'numplays': 'Int32',
            'numplays_month': 'Int32', 'numfans': 'Int32'}
        boardgame_stats = boardgame_stats.astype(boardgame_stats_dtypes, errors='ignore')

        #merge different sources that are at same level togeather 
        boardgame_stats = boardgame_stats.merge(boardgame_stats_polls_recommended, on='objectid', how='left', suffixes=(None, '_Right'))
        boardgame_stats = boardgame_stats.merge(boardgame_stats_polls_best, on='objectid', how='left', suffixes=(None, '_Right'))
        boardgame_details = boardgame_details.merge(boardgame_stats, on='objectid', how='left', suffixes=(None, '_Right'))

        #add the updated date onto each dataframe
        boardgame_details['updated_date'] = today
        boardgame_links['updated_date'] = today
        boardgame_stats_rank['updated_date'] = today
        return boardgame_details, boardgame_links, boardgame_stats_rank

In [4]:
logging.basicConfig(filename='logs\webscraper_{}.log'.format(datetime.today().strftime('%Y%m%d_%H%M%S')), filemode='w',level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
a_boardgamescraper= boardgamescraper('config.yaml', True)     

In [ ]:
page = 1


id_list = a_boardgamescraper.getBoardgameIds(page)

details =[]
stats = []
for row in id_list:
    objectid = row['objectid']
    logging.warning(f'Getting object id : {objectid}')
    data_details = a_boardgamescraper.getBoardgameDetails(objectid)
    if data_details:
        details.append(data_details)
    
    data_stats = a_boardgamescraper.getBoardgameStats(objectid)
    if data_stats:
        stats.append(data_stats)

In [6]:
boardgame_details, boardgame_links, boardgame_ranks = a_boardgamescraper.ParseData(details, stats)

In [8]:
try:
    # Start from last time 
    logging.info(f'Read Page from file')
    with open("lastPage", "r") as inFile:
        page = int(inFile.read())
except:
    logging.error(f'failed to read next page')

page_original = page

while page:
    #get list of id from next page
    logging.info(f'Staring scrapping of page:{page}')
    id_list = a_boardgamescraper.getBoardgameIds(page)
    
    #for each id get the details and stats
    details =[]
    stats = []
    for row in id_list:
        objectid = row['objectid']
        logging.info(f'Getting object id : {objectid}')
        data_details = a_boardgamescraper.getBoardgameDetails(objectid)
        if data_details:
            details.append(data_details)
        
        data_stats = a_boardgamescraper.getBoardgameStats(objectid)
        if data_stats:
            stats.append(data_stats)

    #parse all the data
    boardgame_details, boardgame_links, boardgame_ranks = a_boardgamescraper.ParseData(details, stats)

    a_boardgamescraper.SQLOutputUpsert(boardgame_details,'boardgame_details',['objectid'])
    a_boardgamescraper.SQLOutputUpsert(boardgame_links,'boardgame_links',['gameid','objectid'])
    a_boardgamescraper.SQLOutputUpsert(boardgame_ranks,'boardgame_ranks',['rankobjectid','objectid'])

    page += 1
    with open("lastPage", "w") as outFile:
        outFile.write(str(page))

    if page > page_original + 400:
        break
